In [1]:
import pyuba as uba
import pandas as pd
import numpy as np

In [2]:
payment = pd.read_excel("input/data1.xlsx", sheet_name="payment")
payment.head()

,payment_id,item,payment,buy_date,user_id
0,1,아이템E,20000,2019-12-01,70
1,2,아이템B,5000,2019-12-01,1153
2,3,아이템A,3000,2019-12-01,1210
3,4,아이템A,3000,2019-12-01,1242
4,5,아이템E,20000,2019-12-01,975


In [3]:
sign_up = pd.read_excel("input/data1.xlsx", sheet_name="signup")
sign_up

,user_id,sign_up,last_login
0,1,2019-01-01,2019-06-03
1,2,2019-01-01,2019-11-02
2,3,2019-01-01,2019-01-23
3,4,2019-01-01,2019-04-19
4,5,2019-01-01,2019-06-25
...,...,...,...
10466,10467,2019-12-31,2019-12-31
10467,10468,2019-12-31,2019-12-31
10468,10469,2019-12-31,2019-12-31
10469,10470,2019-12-31,2019-12-31


In [4]:
payment

,payment_id,item,payment,buy_date,user_id
0,1,아이템E,20000,2019-12-01,70
1,2,아이템B,5000,2019-12-01,1153
2,3,아이템A,3000,2019-12-01,1210
3,4,아이템A,3000,2019-12-01,1242
4,5,아이템E,20000,2019-12-01,975
...,...,...,...,...,...
1649,1650,아이템A,3000,2019-12-31,7474
1650,1651,아이템B,5000,2019-12-31,7502
1651,1652,아이템E,20000,2019-12-31,7474
1652,1653,아이템E,20000,2019-12-31,5414


In [5]:
def split_revenue(
    sign_up: pd.DataFrame,
    payment: pd.DataFrame,
    resiual_rate_color: str = "lightgreen",
    payment_rate_color: str = "#ee1f5f",
    arppu_color="lightblue",
) -> pd.DataFrame:
    """
    Show Revenue Table
    :param sign_up: (pd.DataFrame)
                sign_up dataset
    :param payment: (pd.DataFrame)
                payment dataset
    :param resiual_rate_color: (str)
                resiual_rate feature bar color
    :param payment_rate_color: (str)
                payment_rate feature bar color
    :param arppu_color: (str)
                ARPPU feature bar color
    :return: (pd.DataFrame)
            revenue dataframe
    """
    try:
        sign_up["sign_up_month"] = sign_up["sign_up"].dt.month
        sign_up["last_login_month"] = sign_up["last_login"].dt.month

    except AttributeError:
        sign_up["sign_up"] = pd.to_datetime(sign_up["sign_up"])
        sign_up["last_login"] = pd.to_datetime(sign_up["last_login"])
        sign_up["sign_up_month"] = sign_up["sign_up"].dt.month
        sign_up["last_login_month"] = sign_up["last_login"].dt.month

    sign_up_dict = sign_up.groupby("user_id")["sign_up"].first().to_dict()
    payment["sign_up_month"] = payment["user_id"].map(sign_up_dict)
    payment["sign_up_month"] = payment["sign_up_month"].dt.month

    retention = (
        payment.groupby("user_id")
        .agg(
            payment_count=("payment", "count"),
            sales=("payment", "sum"),
            sign_up_month=("sign_up_month", "first"),
        )
        .reset_index()
    )

    retention = (
        retention.groupby("sign_up_month")
        .agg(buyer=("sign_up_month", "count"), sales=("sales", "sum"))
        .reset_index()
    )

    retention["sign_up_number"] = (
        sign_up.groupby("sign_up_month")["user_id"].count().values
    )
    retention["activate"] = (
        sign_up.groupby(["sign_up_month", "last_login_month"])["user_id"]
        .count()
        .unstack()
        .iloc[:, -1]
        .values
    )

    retention["residual_rate"] = (
        retention["activate"] / retention["sign_up_number"]
    ) * 100
    retention["payment_rate"] = (retention["buyer"] / retention["activate"]) * 100
    retention["ARPPU"] = retention["sales"] / retention["buyer"]

    retention = (
        retention.style.format(
            {
                "residual_rate": "{:.0f}%",
                "payment_rate": "{:.0f}%",
                "ARPPU": "{:.0f}",
            }
        )
        .bar(align="mid", subset=["residual_rate"], color=resiual_rate_color)
        .bar(align="mid", subset=["payment_rate"], color=payment_rate_color)
        .bar(align="mid", subset=["ARPPU"], color=arppu_color)
    )

    return retention

In [8]:
retention = split_revenue(sign_up, payment)

In [22]:
retention

,sign_up_month,buyer,sales,sign_up_number,activate,residual_rate,payment_rate,ARPPU
0,1,25,221000,428,59.000000,14%,42%,8840
1,2,39,301000,481,111.000000,23%,35%,7718
2,3,57,496000,545,171.000000,31%,33%,8702
3,4,49,495000,501,184.000000,37%,27%,10102
4,5,58,542000,530,238.000000,45%,24%,9345
5,6,65,597000,530,291.000000,55%,22%,9185
6,7,70,643000,495,308.000000,62%,23%,9186
7,8,72,634000,534,348.000000,65%,21%,8806
8,9,99,1175000,632,473.000000,75%,21%,11869
9,10,128,1299000,737,642.000000,87%,20%,10148


In [ ]:
def mau_plot(activate: Union[pd.Series, np.ndarray, List[float]]) -> Figure:
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=[
                "January",
                "February",
                "March",
                "April",
                "May",
                "June",
                "July",
                "August",
                "September",
                "October",
                "November",
                "December",
            ],
            y=activate,
            mode="lines+markers",  # Line Plot에 마커찍기
            name="lines+markers",
        )
    )
    fig.update_xaxes(tickangle=30)
    
    return fig


In [21]:
from plotly.offline import iplot
import plotly.graph_objects as go
from plotly.graph_objects import Figure
# Create traces
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[
            "January",
            "February",
            "March",
            "April",
            "May",
            "June",
            "July",
            "August",
            "September",
            "October",
            "November",
            "December",
        ],
        y=retention.data["activate"],
        mode="lines+markers",  # Line Plot에 마커찍기
        name="lines+markers",
    )
)
fig.update_xaxes(tickangle=30)
iplot(fig)